In [22]:
import numpy as np
import numpy.linalg as la

#############################
# CONFIGURATION / INPUT DATA
#############################

# Целевая модель улучшения:
TARGET_LEVEL = 10            # Конечный уровень вещи (состояния 0...10, 10 – терминальное, E(10)=0)
NUM_STATES = TARGET_LEVEL + 1

# Исходные данные камней:
arr_lvl = list(range(66, 114))  
arr_vel = [0.017, 0.035, 0.053, 0.071, 0.088, 0.106, 0.124, 0.142, 0.16, 0.177, 
           0.195, 0.213, 0.231, 0.248, 0.266, 0.284, 0.302, 0.32, 0.337, 0.355, 
           0.373, 0.391, 0.408, 0.426, 0.444, 0.462, 0.48, 0.497, 0.515, 0.533, 
           0.551, 0.568, 0.586, 0.604, 0.622, 0.64, 0.657, 0.675, 0.693, 0.711, 
           0.728, 0.746, 0.764, 0.782, 0.8, 0.817, 0.835, 0.85]
arr_price = [27700, 39833, 35233, 30700, 44000, 27800, 30400, 31600, 28250, 29400, 
             37200, 38400, 42000, 55800, 90400, 85000, 88962, 83000, 114500, 247200, 
             278000, 341333, 388000, 554667, 614167, 623333, 616667, 741667, 965000, 
             1658333, 2787037, 2772130, 2850000, 3391667, 3951852, 5848333, 5270000, 
             5166667, 5744444, 6253722, 6150000, 6983333, 7523083, 7584167, 9883333, 
             10685185, 11703704, 13000000]

# Создаем список камней, где каждый камень представляется словарем
stones = []
for i in range(len(arr_lvl)):
    stones.append({
        'stone_level': arr_lvl[i],
        'price': arr_price[i],
        'p_success': arr_vel[i]   # вероятность успеха уже задана как число от 0.017 до 0.85
    })

# Параметры динамики улучшения:
# При успехе: распределение прироста уровней (на сколько повышается вещь)
SUCCESS_DISTRIBUTION = [(1, 0.90), (2, 0.07), (3, 0.03)]  # сумма вероятностей = 1

# При неудаче: если уровень вещи равен 0 – не меняется, иначе снижается на 1 уровень
def failure_distribution(state):
    if state == 0:
        return [(0, 1.0)]
    else:
        return [(-1, 1.0)]

# Параметры алгоритма Policy Iteration:
MAX_POLICY_ITER = 100000      # максимальное число итераций по политике
TOLERANCE = 1e-12            # критерий сходимости
INITIAL_COST = 1000.0       # начальное приближение для E(state) для state 0...9

#############################
# END OF CONFIGURATION
#############################

# Для решения будем рассматривать состояния вещи от 0 до TARGET_LEVEL-1 (для TARGET_LEVEL E=0)
num_non_terminal = TARGET_LEVEL  # состояний, для которых решаем уравнения

# Инициализируем начальную политику: для всех состояний выбираем камень с наибольшей вероятностью успеха
initial_policy_index = np.argmax([s['p_success'] for s in stones])
policy = [initial_policy_index] * num_non_terminal

# Начальное приближение для функции стоимости (E) для состояний 0..TARGET_LEVEL-1
E_vals = np.full(num_non_terminal, INITIAL_COST, dtype=float)

def Q_value(state, stone, E_vals):
    """
    Рассчитывает Q-значение для состояния state при использовании данного камня.
    Это: цена камня + p_success * (ожидаемая стоимость при успехе) + (1-p_success) * (ожидаемая стоимость при неудаче)
    """
    cost = stone['price']
    p = stone['p_success']
    
    # Ожидаемая стоимость при успехе:
    success_cost = 0.0
    for delta, prob in SUCCESS_DISTRIBUTION:
        next_state = min(state + delta, TARGET_LEVEL)
        if next_state < TARGET_LEVEL:
            success_cost += prob * E_vals[next_state]
        # Если next_state == TARGET_LEVEL, E=0 (терминальное состояние)
    
    # Ожидаемая стоимость при неудаче:
    fail_cost = 0.0
    for delta, prob in failure_distribution(state):
        next_state = state + delta
        next_state = max(0, min(next_state, TARGET_LEVEL))
        if next_state < TARGET_LEVEL:
            fail_cost += prob * E_vals[next_state]
    return cost + p * success_cost + (1 - p) * fail_cost

# Policy Iteration: оценка и улучшение политики
policy_stable = False
iteration = 0

while not policy_stable and iteration < MAX_POLICY_ITER:
    iteration += 1
    
    # ---------------------------
    # Шаг 1: Оценка политики (Policy Evaluation)
    # Составляем систему линейных уравнений для состояний 0 .. TARGET_LEVEL-1.
    A = np.zeros((num_non_terminal, num_non_terminal))
    b = np.zeros(num_non_terminal)
    
    for state in range(num_non_terminal):
        stone = stones[policy[state]]
        cost = stone['price']
        p = stone['p_success']
        
        # Основное уравнение: E(state) = cost + p * (ожидаемая стоимость при успехе) + (1-p) * (ожидаемая стоимость при неудаче)
        # Переносим все слагаемые с E(...) в левую часть.
        A[state, state] = 1.0  # коэффициент при E(state)
        
        # Учет успеха: для каждого delta из SUCCESS_DISTRIBUTION
        for delta, prob in SUCCESS_DISTRIBUTION:
            next_state = min(state + delta, TARGET_LEVEL)
            if next_state < TARGET_LEVEL:
                A[state, next_state] -= p * prob
            # Если next_state == TARGET_LEVEL, E = 0, поэтому ничего не добавляем.
        
        # Учет неудачи:
        # Если state == 0, то при неудаче остаемся на 0; иначе E(state-1)
        if state == 0:
            A[state, 0] -= (1 - p)
        else:
            A[state, state - 1] -= (1 - p)
        
        b[state] = cost
    
    try:
        E_new = la.solve(A, b)
    except la.LinAlgError:
        print("Ошибка: не удалось решить систему линейных уравнений при оценке политики.")
        break

    max_diff = np.max(np.abs(E_new - E_vals))
    E_vals = E_new.copy()
    
    # ---------------------------
    # Шаг 2: Улучшение политики (Policy Improvement)
    policy_stable = True
    for state in range(num_non_terminal):
        current_action = policy[state]
        current_Q = Q_value(state, stones[current_action], E_vals)
        best_action = current_action
        # Перебор всех возможных камней для выбора наилучшего по Q-значению
        for i, stone in enumerate(stones):
            q_val = Q_value(state, stone, E_vals)
            if q_val < current_Q:
                current_Q = q_val
                best_action = i
        if best_action != policy[state]:
            policy[state] = best_action
            policy_stable = False

    if max_diff < TOLERANCE:
        break

#############################
# OUTPUT RESULTS
#############################

print("Оптимальная политика для каждого уровня вещи (состояния 0..9):")
for state in range(num_non_terminal):
    chosen_stone = stones[policy[state]]
    print(f"Уровень {state}: использовать камень уровня {chosen_stone['stone_level']} "
          f"(Цена: {chosen_stone['price']}, Шанс успеха: {chosen_stone['p_success']:.3f})")

print("\nТаблица альтернатив (топ-5 вариантов по Q-значению) для каждого состояния:")
for state in range(num_non_terminal):
    alternatives = []
    for stone in stones:
        alternatives.append((stone, Q_value(state, stone, E_vals)))
    alternatives.sort(key=lambda x: x[1])
    print(f"\nУровень вещи {state}:")
    header = f"{'Камень':>10} | {'Цена':>8} | {'Шанс':>8} | {'Q-значение':>15}"
    print(header)
    print("-" * len(header))
    for stone, q_val in alternatives[:8]:
        print(f"{stone['stone_level']:10} | {stone['price']:8} | {stone['p_success']:8.3f} | {q_val:15.0f}")

print("\nОжидаемые затраты (E) для состояний:")
for state in range(num_non_terminal):
    print(f"Уровень {state+1}: {E_vals[state]:.2f}")
# print("Уровень 10: 0.00")


Оптимальная политика для каждого уровня вещи (состояния 0..9):
Уровень 0: использовать камень уровня 75 (Цена: 29400, Шанс успеха: 0.177)
Уровень 1: использовать камень уровня 83 (Цена: 83000, Шанс успеха: 0.320)
Уровень 2: использовать камень уровня 83 (Цена: 83000, Шанс успеха: 0.320)
Уровень 3: использовать камень уровня 84 (Цена: 114500, Шанс успеха: 0.337)
Уровень 4: использовать камень уровня 92 (Цена: 616667, Шанс успеха: 0.480)
Уровень 5: использовать камень уровня 92 (Цена: 616667, Шанс успеха: 0.480)
Уровень 6: использовать камень уровня 93 (Цена: 741667, Шанс успеха: 0.497)
Уровень 7: использовать камень уровня 93 (Цена: 741667, Шанс успеха: 0.497)
Уровень 8: использовать камень уровня 93 (Цена: 741667, Шанс успеха: 0.497)
Уровень 9: использовать камень уровня 94 (Цена: 965000, Шанс успеха: 0.515)

Таблица альтернатив (топ-5 вариантов по Q-значению) для каждого состояния:

Уровень вещи 0:
    Камень |     Цена |     Шанс |      Q-значение
------------------------------------

##### 